### This Notebook shows how a **Backer** can writte KERI Key Events to Cardano blockchain and how a **Watcher** can retrieve the full Key Event Log (KEL) from the blockchain.
This code use Blockfrost API to intercat with Cardano Nodes

In [ ]:
from blockfrost import BlockFrostApi, ApiError, ApiUrls
from pycardano import * 
import os

First create a ProjectId in https://blockfrost.io for the network of your choice. In this case we are using the _Preview_ network.
Put your projectId in an evironmental variable or paste it in the code bellow.

In [309]:
blockfrostProjectId = os.getenv('BLOCKFROST_PROJECT_ID')
blockfrostProjectId="previewapifaDDKsMZE7asmrcG8W3zbRE1pojXY"
api = BlockFrostApi(
    project_id=blockfrostProjectId,
    base_url=ApiUrls.preview.value
)
network = Network.TESTNET
context = BlockFrostChainContext(blockfrostProjectId,network, ApiUrls.preview.value)

Generate payment and stake key pairs. You can Save and Load on this disk.

In [ ]:
payment_key_pair = PaymentKeyPair.generate()
payment_signing_key = payment_key_pair.signing_key
payment_verification_key = payment_key_pair.verification_key

stake_key_pair = StakeKeyPair.generate()
stake_signing_key = stake_key_pair.signing_key
stake_verification_key = stake_key_pair.verification_key

In [298]:
# Save keys
payment_signing_key.save("payment.skey")
payment_verification_key.save("payment.vkey")
stake_signing_key.save("stake.skey")
stake_verification_key.save("stake.vkey")

In [ ]:
# Load keys
payment_signing_key = PaymentSigningKey.load("payment.skey")
payment_verification_key = PaymentVerificationKey.load("payment.vkey")
stake_signing_key = StakeSigningKey.load("stake.skey")
stake_verification_key = StakeVerificationKey.load("stake.vkey")

Generate the Stake Address and the Spending Address that the Backer will use to submit transactions with metadata to Cardano.
You can fund the Spending Address with test ADA at https://docs.cardano.org/cardano-testnet/tools/faucet"

In [316]:
spending_addr = Address(payment_verification_key.hash(), stake_verification_key.hash(), network=network)
stake_addr = Address(payment_part=None, staking_part=stake_verification_key.hash(), network=network)
print("Stake address:", stake_addr.encode())
print("Spending Address:", spending_addr.encode())

Stake address: stake_test1ur3vs3x0xl97tt2gau5wknmxve0jlmsqn2l2dqrrt40tgzsmhat72
Spending Address: addr_test1qrtvrpe58csevt7esxqjv4rpvm5q30paghtlse2pj3xn8g0zepzv7d7tukk53megad8kvejl9lhqpx4756qxxh27ks9qqxkex3
Fund address above in https://docs.cardano.org/cardano-testnet/tools/faucet


Using Blockfrost API you can query ADA balances and UTXOs

In [357]:
# Check address balance
address = api.address(
        address=spending_addr.encode())
for amount in address.amount:
    print(amount.quantity, amount.unit)

99319472 lovelace


In [362]:
# Check wallet balance
addresses = api.account_addresses(stake_addr.encode())
for addr in addresses:
    addrR = api.address(addr.address)
    for amount in addrR.amount:
        print(amount.quantity, amount.unit,":", addrR.address)


99319472 lovelace : addr_test1qrtvrpe58csevt7esxqjv4rpvm5q30paghtlse2pj3xn8g0zepzv7d7tukk53megad8kvejl9lhqpx4756qxxh27ks9qqxkex3


In [365]:
# Get UTXOs
utxos = api.address_utxos(spending_addr.encode())
print(utxos)

[Namespace(tx_hash='31f1980ab3a9ac0aca5ad687ffcc0f8ff9e84d6ae02b111500adc7344cf11a5a', tx_index=0, output_index=0, amount=[Namespace(unit='lovelace', quantity='1000000')], block='80468a6dcb2ce937ac6b319e4203037d8ceb34a619c4db104441651137c34be2', data_hash=None, inline_datum=None, reference_script_hash=None), Namespace(tx_hash='6428ac788501671e81fc2c6427e256d9360bdc3400febbef1d61baa06fe2528d', tx_index=0, output_index=0, amount=[Namespace(unit='lovelace', quantity='1000000')], block='63d852fca752fa86afbe400728a85e577057d8673eb016124a2fb911cc3b8f35', data_hash=None, inline_datum=None, reference_script_hash=None), Namespace(tx_hash='6428ac788501671e81fc2c6427e256d9360bdc3400febbef1d61baa06fe2528d', tx_index=1, output_index=1, amount=[Namespace(unit='lovelace', quantity='97319472')], block='63d852fca752fa86afbe400728a85e577057d8673eb016124a2fb911cc3b8f35', data_hash=None, inline_datum=None, reference_script_hash=None)]


Build and sign a transaction. The Key Event is added as transaction metadata.

In [377]:
#Build and Sign transaction
builder = TransactionBuilder(context)
builder.add_input_address(spending_addr)
builder.add_output(TransactionOutput(spending_addr,Value.from_primitive([1000000])))
builder.auxiliary_data = AuxiliaryData(Metadata(
            { 
                1: {
                    "v": "KERI10JSON000159_",
                    "t": "icp",
                    "d": "EDRc4HIqxKvCeafMxSMJM2tuQfkERkgtzUAAVLdk-bhj",
                    "i": "EDRc4HIqxKvCeafMxSMJM2tuQfkERkgtzUAAVLdk-bhj",
                    "s": "0",
                    "kt": "1",
                    "k": [
                    "DNnrwNUFl070TGU8wlYSZZ0WZ-V5WbDqXnBv-HkXjQ4x"
                    ],
                    "nt": "1",
                    "n": [
                    "EJ2BHzcravgL1BfgEbNo8V181WNvKprybAkae4QSLaG1"
                    ],
                    "bt": "1",
                    "b": [
                    "BBilc4-L3tFUnfM_wJr4S4OJanAv_VmF_dJNN6vkf2Ha"
                    ],
                    "c": [],
                    "a": []
                }
            }
        )
    )
signed_tx = builder.build_and_sign([payment_signing_key], change_address=spending_addr)
print(signed_tx.id)

e58c55a566b3665fa98af0888ed2544d082ea3d5ea6724c6fba231a731fbe86f


In [367]:
#Submit transaction
context.submit_tx(signed_tx.to_cbor())

In [375]:
#Check transaction (need time to be published)
tx = api.transaction(signed_tx.id)
print(tx)

Namespace(hash='e4db3f05bc35d6245d10f9ca943d9041c5e440b31105cee9caee5c2d3d06fa3e', block='6d31e7bbddb12a7298e0333f998ae4ac561d5ff4f8814dd23e195ee4e289e174', block_height=303965, block_time=1666575275, slot=6572075, index=0, output_amount=[Namespace(unit='lovelace', quantity='97149703')], fees='169769', deposit='0', size=326, invalid_before=None, invalid_hereafter=None, utxo_count=3, withdrawal_count=0, mir_cert_count=0, delegation_count=0, stake_cert_count=0, pool_update_count=0, pool_retire_count=0, asset_mint_or_burn_count=0, redeemer_count=0, valid_contract=True)


In [340]:
#Checl transaction metadata
metadata = api.transaction_metadata(signed_tx.id)
print(metadata)

[Namespace(label='221', json_metadata=Namespace(KEL='1'))]


In [376]:
# Get all metadata from the address
txs = api.address_transactions(spending_addr.encode())
for tx in txs:
    print(api.transaction_metadata(tx.tx_hash))

[]
[]
[Namespace(label='221', json_metadata=Namespace(bla='bla'))]
[Namespace(label='221', json_metadata=Namespace(KEL='1'))]
[Namespace(label='2', json_metadata=Namespace(KEL='2'))]
[Namespace(label='3', json_metadata=Namespace(KEL='3'))]
